In [8]:
from tkinter import *
import tkinter as tk
from PIL import ImageTk, Image
import cv2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import tensorflow_hub as hub
import numpy as np
import imutils
import time
import cv2
import tkinter
import os
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import datetime as dt


def isbright(image, dim=10, thresh=0.5):
    # Resize image to 10x10
    image = cv2.resize(image, (dim, dim))
    # Convert color space to LAB format and extract L channel
    L, A, B = cv2.split(cv2.cvtColor(image, cv2.COLOR_BGR2LAB))
    # Normalize L channel by dividing all pixel values with maximum pixel value
    L = L/np.max(L)
    # Return True if mean is greater than thresh else False
    return np.mean(L) > thresh


    
 

def stopcam():
    cap.release()
    l3.config(text = "Camera Closed")


def detect_and_predict_mask(frame, faceNet, maskNet):
        (h, w) = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),(104.0, 177.0, 123.0))
        faceNet.setInput(blob)
        detections = faceNet.forward()
        print(detections.shape)
        faces = []
        global score
        global score_lite
        locs = []
        preds = []
      
        for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                (startX, startY) = (max(0, startX), max(0, startY))
                (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
                face = frame[startY:endY, startX:endX]
                face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
                face = cv2.resize(face, (224, 224))
                face = img_to_array(face)
                face = preprocess_input(face)
                faces.append(face)
                locs.append((startX, startY, endX, endY))
        if len(faces) > 0:
                
                faces = np.array(faces, dtype="float32")
                preds = maskNet.predict(faces, batch_size=32)
                score_lite=preds
                score = np.argmax(preds)
              
        return (locs, preds)
def helloCallBack():
        global cap
        cap=cv2.VideoCapture(0)
        prototxtPath = r"face_detector\deploy.prototxt"
        weightsPath = r"face_detector\res10_300x300_ssd_iter_140000.caffemodel"
        faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)
        maskNet = tf.keras.models.load_model(("mask_MobileNet_detector.model"), 
                                     custom_objects={'KerasLayer':hub.KerasLayer})
        
        
        while True:
            img=cap.read()[1]
            l5.config(text="Good Brightness") if isbright(img) else l5.config(text=" Low Brightness")
            obj=img
            (locs, preds) = detect_and_predict_mask(obj, faceNet, maskNet)
            for (box, pred) in zip(locs, preds):
                (startX, startY, endX, endY) = box
                pred=score 
                if score==0:
                    label=("cloth")
                    label = "{}: {:.2f}%".format(label,100 * np.max(score_lite))
                    color = (0, 225, 0)
                    l3.config(text = "cloth mask",bg='green')
                elif score == 1:
                    label=("n95")
                    label = "{}: {:.2f}%".format(label,100 * np.max(score_lite))
                    color = (0, 225, 0)
                    l3.config(text = "n95 mask",bg='green')
                elif score == 2:
                    label=("n95v")
                    label = "{}: {:.2f}%".format(label,100 * np.max(score_lite))
                    color = (0, 225, 0)
                    l3.config(text = "n95v Mask",bg='green')
                elif  score==3:
                    label=("No Mask")
                    label = "{}: {:.2f}%".format(label,100 * np.max(score_lite))
                    color = (0, 0, 255)
                    l3.config(text = "No mask",bg='red')
                else:
                    label=("srg")
                    l3.config(text = "Sargical Mask",bg='green')
                    label = "{}: {:.2f}%".format(label,100 * np.max(score_lite))
                    color = (0, 225, 0)
                   
            
            cv2.putText(obj, label, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(obj, (startX, startY), (endX, endY), color, 2)
            obj=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            obj=ImageTk.PhotoImage(Image.fromarray(obj))
            l1['image']=obj
            win.update()
        return cap
        

            
            
    
#Create an instance of tkinter frame
win= Tk()
global my_text
win.configure(bg='#F1EB9C')
win.title('Mask Detection System')
p1 = PhotoImage(file = 'bt1.png')
#win.iconphoto(False, p1)
#Set the geometry of frame
win.geometry("1250x750")
btn1 = Button(win,bg='#540b0e', text = 'open Your Camera !',foreground='white',height= 3, width=20,border=5, command=helloCallBack)
btn1.grid(row=1,column=0)
date = dt.datetime.now()
label = Label(win, bg="#ffbf69",border=5,foreground='#3a0ca3', text=f"{date:%A, %B %d, %Y}", font="Calibri, 10",height= 3, width=20)
label.grid(row=0,column=0)
btn1 = Button(win,bg='#a7c957',foreground='white' ,border=5,text = 'Stop Your Camera !',height= 3, width=20, command=stopcam)
btn1.grid(row=1,column=2)
l2=Label(win,text='MASK STATUS',font=" 6", bg='#CCFF00',height= 3, width=20,)
l2.grid(row=0,column=3) 
l3=Label(win,bg='#9b5de5',font=" 6",height= 3, width=20)
l3.grid(row=1,column=3)
l4=Label(win,text='Brightness STATUS',font=" 6", bg='#8338ec',height= 3, width=20)
l4.grid(row=0,column=4) 
l5=Label(win,bg='#a9def9',font=" 6",height= 3, width=20)
l5.grid(row=1,column=4)
f1 = Frame(win,)  
f1.grid(row=4,column=0)  
l1 = Label(f1,)  
l1.grid(row=4,column=0) 


win.mainloop()

(1, 1, 200, 7)
1/1 [==============================] - 1s 588ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 67ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 65ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 68ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 64ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 73ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 62ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 82ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 71ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 81ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 75ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 81ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 76ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 62ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 70ms

1/1 [==============================] - 0s 61ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 59ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 67ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 61ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 65ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 76ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 57ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 59ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 62ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 61ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 60ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 61ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 58ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 62ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 60ms/step
(1, 1, 200

1/1 [==============================] - 0s 77ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 64ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 63ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 66ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 69ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 61ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 69ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 66ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 66ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 58ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 59ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 58ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 58ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 69ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 60ms/step
(1, 1, 200

1/1 [==============================] - 0s 64ms/step
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
1/1 [==============================] - 0s 61ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 60ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 62ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 77ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 71ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 62ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 56ms/step
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
1/1 [==============================] - 0s 62

1/1 [==============================] - 0s 57ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 62ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 70ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 60ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 68ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 61ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 77ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 67ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 61ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 71ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 64ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 66ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 62ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 59ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 62ms/step
(1, 1, 200

1/1 [==============================] - 0s 56ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 70ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 63ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 55ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 81ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 59ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 55ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 57ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 61ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 61ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 60ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 69ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 75ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 62ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 59ms/step
(1, 1, 200

1/1 [==============================] - 0s 68ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 77ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 61ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 72ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 67ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 58ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 61ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 60ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 60ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 63ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 58ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 67ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 58ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 56ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 64ms/step
(1, 1, 200

1/1 [==============================] - 0s 77ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 68ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 71ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 64ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 59ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 63ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 59ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 63ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 63ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 60ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 63ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 60ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 60ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 67ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 60ms/step
(1, 1, 200

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Super\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Super\AppData\Local\Temp\ipykernel_3104\979652970.py", line 88, in helloCallBack
    l5.config(text="Good Brightness") if isbright(img) else l5.config(text=" Low Brightness")
  File "C:\Users\Super\AppData\Local\Temp\ipykernel_3104\979652970.py", line 25, in isbright
    image = cv2.resize(image, (dim, dim))
cv2.error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

